In [11]:
# Import main DataFrames from the data folder
import matplotlib.pyplot as plt
import pandas as pd

imported_dfs = {}
for name in ["final_trkData", "tcqData", "notifCount", "MWctr", "preQDat", "postQDat"]:
    imported_dfs[name] = pd.read_csv(f"data/{name}.csv", index_col=0)
    print(f"Imported {name} from data/{name}.csv")

# Optionally, unpack to variables:
final_trkData = imported_dfs["final_trkData"]
tcqData = imported_dfs["tcqData"]
notifCount = imported_dfs["notifCount"]
MWctr = imported_dfs["MWctr"]
preQDat = imported_dfs["preQDat"]
postQDat = imported_dfs["postQDat"]

Imported final_trkData from data/final_trkData.csv
Imported tcqData from data/tcqData.csv
Imported notifCount from data/notifCount.csv
Imported MWctr from data/MWctr.csv
Imported preQDat from data/preQDat.csv
Imported postQDat from data/postQDat.csv


In [12]:
len(tcqData['expiwellID'].unique())

141

In [13]:

#get the length and colnames
#get the length and colnames
print(f"Shape: {tcqData.shape}")
print(f"Columns: {tcqData.columns.tolist()}")
tcqDataMaster=tcqData.copy()
#ok so its already cleaned
#but we shoudl subset for the participatns whose mind was actually wandering, so ontask >1
tcqData=tcqData[tcqData['onTask']>1]
#specficiically for the CFA, lets not include factors not present in the original
tcq_cols = ['visImag', 'inSpeech', 'intent', 'struct', 'real', 'concAbs', 'import', 'relGoal','oftMind', 'emotVal',]
#drop rows with NaN in any of the tcq_cols
tcqData = tcqData.dropna(subset=tcq_cols)
#lost like 10 rows
within = tcqData.copy()
between = tcqData.copy()
#mean center within participants
within[tcq_cols] = within.groupby("Participant ID")[tcq_cols].transform(lambda x: x - x.mean())
#mean center between participants
#between[tcq_cols] = between.groupby("Participant ID")[tcq_cols].mean()

between = between.groupby("Participant ID")[tcq_cols].mean() - between[tcq_cols].mean()


Shape: (4773, 21)
Columns: ['Start Date', 'End Date', 'Time Scheduled', 'Duration (in seconds)', 'expiwellID', 'Participant ID', 'onTask', 'visImag', 'inSpeech', 'intent', 'struct', 'real', 'concAbs', 'import', 'relGoal', 'fullDev', 'oftMind', 'emotVal', 'tempOr', 'funct', 'functOth']


In [14]:
len(tcqData['expiwellID'].unique())

138

In [15]:

#between[tcq_cols] = between[tcq_cols] - between[tcq_cols].mean()


import pandas as pd
import numpy as np
import statsmodels.api as sm

def icc_random_intercept(df, pid_col, item_col):
    """
    ICC for a single item with episodes nested within participant:
      y_ij = mu + u_j + e_ij
      ICC = Var(u) / (Var(u) + Var(e))

    Returns: icc, var_between, var_within, n_obs, n_groups
    """
    dat = df[[pid_col, item_col]].dropna().copy()
    dat[item_col] = pd.to_numeric(dat[item_col], errors="coerce")
    dat = dat.dropna()

    n_obs = dat.shape[0]
    n_groups = dat[pid_col].nunique()

    # Need at least 2 groups and >1 observation per some groups to estimate variance components
    if n_groups < 2 or n_obs < 3:
        return np.nan, np.nan, np.nan, n_obs, n_groups

    # Random intercept model: y ~ 1 + (1|pid)
        # MixedLM in statsmodels uses "groups" for random intercept by default.
    model = sm.MixedLM(endog=dat[item_col], exog=np.ones((n_obs, 1)), groups=dat[pid_col])
    try:
        fit = model.fit(reml=True, method="cg", disp=False)
        if not fit.converged:
             print(f"Warning: Model for '{item_col}' did not converge.")
    except Exception as e:
        print(f"Error fitting model for '{item_col}': {e}")
        return np.nan, np.nan, np.nan, n_obs, n_groups

    # Extract variance components
    var_between = float(fit.cov_re.iloc[0, 0])      # Var(u)
    var_within = float(fit.scale)                  # Var(e)

    icc = var_between / (var_between + var_within) if (var_between + var_within) > 0 else np.nan
    return icc, var_between, var_within, n_obs, n_groups


def icc_table(df, pid_col="pid", tcq_cols=[]):

    rows = []
    for col in tcq_cols:
        icc, vb, vw, n_obs, n_groups = icc_random_intercept(df, pid_col, col)
        rows.append({
            "item": col,
            "ICC": icc,
            "Var_between": vb,
            "Var_within": vw,
            "n_obs": n_obs,
            "n_participants": n_groups
        })

    out = pd.DataFrame(rows).sort_values("ICC", ascending=False)
    return out



iccs = icc_table(tcqData, pid_col="Participant ID", tcq_cols=tcq_cols)
print(iccs.to_string(index=False))



    item      ICC  Var_between  Var_within  n_obs  n_participants
 visImag 0.394163     1.391887    2.139358   2250             138
inSpeech 0.323456     1.059262    2.215565   2250             138
 oftMind 0.316858     0.895448    1.930573   2250             138
  intent 0.304032     0.982153    2.248273   2250             138
  struct 0.291522     0.835202    2.029768   2250             138
    real 0.287290     0.792455    1.965922   2250             138
 concAbs 0.280046     0.828559    2.130095   2250             138
 relGoal 0.271129     0.997444    2.681407   2250             138
  import 0.228576     0.667857    2.253961   2250             138
 emotVal 0.158683     0.308471    1.635473   2250             138


In [16]:


# Recode the 'tempOr' column: Past (1) -> -1; Present (2) -> 0; Future (3) -> 1; No specific time (4) -> 0
tempOr_recode_map = {1: -1, 2: 0, 3: 1, 4: 0}
tcqData['tempOr'] = tcqData['tempOr'].replace(tempOr_recode_map)
print("Recoded 'tempOr' column. Displaying a sample:")
display(tcqData['tempOr'].value_counts(normalize=True).sort_index() * 100)
# --- Temporal Orientation Distribution Across Participants ---
""" 
# Group by participant and get the normalized value counts for 'tempOr'
participant_tempOr_dist = tcqData.groupby('expiwellID')['tempOr'].value_counts(normalize=True).unstack(fill_value=0) * 100

# It's possible a participant never used a certain category, so let's ensure all 4 columns exist
for i in range(1, 5):
    if i not in participant_tempOr_dist.columns:
        participant_tempOr_dist[i] = 0

# Rename columns for clarity based on the survey codes: 1=Past, 2=Present, 3=Future, 4=No specific time
participant_tempOr_dist = participant_tempOr_dist.rename(columns={
    1.0: 'Past (%)',
    2.0: 'Present (%)',
    3.0: 'Future (%)',
    4.0: 'No Specific Time (%)'
})


print("--- Distribution of Temporal Orientation Percentages Across Participants ---")
print("\nExample percentages for the first 5 participants:")
display(participant_tempOr_dist.head())

print("\nSummary statistics for these percentages across all participants:")
print("This shows the average (mean), spread (std), and range (min/max) of temporal focus.")
display(participant_tempOr_dist.describe())


# --- Plot for "No Specific Time" Proportion ---
no_time_props = participant_tempOr_dist['No Specific Time (%)'].sort_values(ascending=False)

plt.figure(figsize=(20, 6))
plt.bar(no_time_props.index, no_time_props.values, color='skyblue')
plt.title('Proportion of "No Specific Time" Mind-Wandering Episodes per Participant', fontsize=16)
plt.xlabel('Participant ExpiWell ID', fontsize=12)
plt.ylabel('Percentage of Episodes (%)', fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# --- Plot for "Past" Proportion ---
past_props = participant_tempOr_dist['Past (%)'].sort_values(ascending=False)

plt.figure(figsize=(20, 6))
plt.bar(past_props.index, past_props.values, color='coral')
plt.title('Proportion of "Past" Mind-Wandering Episodes per Participant', fontsize=16)
plt.xlabel('Participant ExpiWell ID', fontsize=12)
plt.ylabel('Percentage of Episodes (%)', fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- Plot for "Present" Proportion ---
present_props = participant_tempOr_dist['Present (%)'].sort_values(ascending=False)

plt.figure(figsize=(20, 6))
plt.bar(present_props.index, present_props.values, color='lightgreen')
plt.title('Proportion of "Present" Mind-Wandering Episodes per Participant', fontsize=16)
plt.xlabel('Participant ExpiWell ID', fontsize=12)
plt.ylabel('Percentage of Episodes (%)', fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- Plot for "Future" Proportion ---
future_props = participant_tempOr_dist['Future (%)'].sort_values(ascending=False)

plt.figure(figsize=(20, 6))
plt.bar(future_props.index, future_props.values, color='lightblue')
plt.title('Proportion of "Future" Mind-Wandering Episodes per Participant', fontsize=16)
plt.xlabel('Participant ExpiWell ID', fontsize=12)
plt.ylabel('Percentage of Episodes (%)', fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- Proportional Stacked Bar Chart of Temporal Orientations ---

# 1. Calculate the proportion of each temporal orientation for each participant
participant_tempOr_props = tcqData.groupby('expiwellID')['tempOr'].value_counts(normalize=True).unstack(fill_value=0)

# 2. Ensure all four categories exist as columns, even if unused
for i in range(1, 5):
    if i not in participant_tempOr_props.columns:
        participant_tempOr_props[i] = 0

# 3. Sort participants by the proportion of "No Specific Time" (tempOr == 4) episodes
sorted_proportions = participant_tempOr_props.sort_values(by=4, ascending=True)

# 4. Rename columns for a clear legend
sorted_proportions = sorted_proportions.rename(columns={
    1: 'Past',
    2: 'Present',
    3: 'Future',
    4: 'No Specific Time'
})

# 5. Reorder columns for a logical stack in the plot
sorted_proportions = sorted_proportions[['Past', 'Present', 'Future', 'No Specific Time']]

# 6. Create the stacked bar plot
ax = sorted_proportions.plot(
    kind='bar',
    stacked=True,
    figsize=(20, 10),
    color=['coral', 'lightgreen', 'lightblue', 'skyblue'],
    width=0.8
)

# 7. Customize the plot for clarity
ax.set_title('Proportion of Temporal Orientations per Participant', fontsize=18)
ax.set_xlabel('Participant ExpiWell ID (Sorted by "No Specific Time" Proportion)', fontsize=14)
ax.set_ylabel('Proportion of Mind-Wandering Episodes', fontsize=14)
ax.tick_params(axis='x', rotation=90, labelsize=8)
ax.tick_params(axis='y', labelsize=10)
ax.legend(title='Temporal Orientation', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_ylim(0, 1) # Proportions sum to 1
plt.tight_layout(rect=[0, 0, 0.9, 1]) # Adjust layout to make space for legend
plt.show()

# --- Analyze Trade-offs in Temporal Orientation ---

# To understand what "No Specific Time" trades off with, we can calculate
# the correlation matrix of the proportions of each temporal category.
# A strong negative correlation indicates that as one proportion goes up, the other goes down.
proportion_correlations = sorted_proportions.corr()

print("\n--- Correlation Matrix of Temporal Orientation Proportions ---")
print("This matrix shows how the proportion of each category relates to the others across participants.")
print("A strong negative value means that as one category's proportion increases, the other's tends to decrease.")
display(proportion_correlations)

# Find the strongest negative correlation with 'No Specific Time'
trade_off = proportion_correlations['No Specific Time'].sort_values().iloc[0]
trade_off_category = proportion_correlations['No Specific Time'].sort_values().index[0]

print(f"\nThe strongest trade-off for 'No Specific Time' is with '{trade_off_category}' (r = {trade_off:.2f}).")
print(f"This suggests that participants who have a higher proportion of 'No Specific Time' thoughts tend to have a correspondingly lower proportion of '{trade_off_category}' thoughts.")
 """

tcq_cols = ['visImag', 'inSpeech', 'intent', 'struct', 'real', 'concAbs', 'import', 'relGoal', 'fullDev', 'oftMind', 'emotVal', 'tempOr']

#this is all the factors except funct and functOth, as opposed to the 10 used in the
# cfa
between = tcqData.groupby("expiwellID")[tcq_cols].mean()



# Calculate and display the between-participant correlation matrix
between_corr = between[tcq_cols].corr()

print("Between-Participant Correlation Matrix:")
display(between_corr)

# Calculate and display the within-participant correlation matrix
within_corr = within[tcq_cols].corr()

print("\nWithin-Participant Correlation Matrix:")
display(within_corr)
from semopy import Model, calc_stats

model_desc = """
Format =~ 1*visImag + -1*inSpeech
Relevance =~ import + relGoal + oftMind
Realism =~ real + concAbs
Structuration =~ struct + intent

# allow factor correlations
Format ~~ Relevance
Format ~~ Realism
Format ~~ Structuration
Relevance ~~ Realism
Relevance ~~ Structuration
Realism ~~ Structuration
"""
def fit_cfa(data, model_desc):
    mod = Model(model_desc)
    mod.fit(data)
    stats = calc_stats(mod)
    
    # Print the stats DataFrame to see available keys
    print("--- SEMOPY Stats Output ---")
    print(stats)
    print("---------------------------")

    fit = {
        "n": data.shape[0],
        "chi2": stats.loc["Value", "chi2"],
        "df": stats.loc["Value", "DoF"],
        "p": stats.loc["Value", "chi2 p-value"],
        "cfi": stats.loc["Value", "CFI"],
        "tli": stats.loc["Value", "TLI"],
        "rmsea": stats.loc["Value", "RMSEA"],
        "srmr": "Not available" # SRMR is not in the output
    }
    est = mod.inspect(std_est=True)
    return mod, fit, est

# Within CFA
Xw = within[tcq_cols].dropna()
print("Columns passed to CFA model:", Xw.columns.tolist())

mod_w, fit_w, est_w = fit_cfa(Xw, model_desc)

print("WITHIN-LEVEL CFA fit:", fit_w)

# Standardized loadings
load_w = est_w[(est_w["op"] == "~") & (est_w["lval"].isin(tcq_cols))]
print(load_w[["lval","rval","Est. Std","p-value"]].sort_values(["rval","lval"]))

# --- Between-Level CFA ---

# Define the between-level model based on the provided description
model_desc_b = """
# Factor 1: Imagery & Structure
ImageryStruct =~ visImag + emotVal + intent + struct

# Factor 2: Relevance
Relevance =~ import + relGoal + oftMind

# Factor 3: Realism
Realism =~ real + concAbs


"""

# Prepare the between-level data
# Note: 'inSpeech' is in tcq_cols but not in the model, semopy will handle this.
Xb = between[tcq_cols].dropna()

print("\n--- Running Between-Level CFA ---")
# Fit the CFA model
mod_b, fit_b, est_b = fit_cfa(Xb, model_desc_b)

print("\nBETWEEN-LEVEL CFA fit:", fit_b)

# Display standardized loadings
print("\nStandardized Loadings (Between-Level):")
load_b = est_b[(est_b["op"] == "~") & (est_b["lval"].isin(tcq_cols))]
print(load_b[["lval","rval","Est. Std","p-value"]].sort_values(["rval","lval"]))

Recoded 'tempOr' column. Displaying a sample:


-1.0     7.022222
 0.0    54.577778
 1.0    38.400000
Name: tempOr, dtype: float64

Between-Participant Correlation Matrix:


,visImag,inSpeech,intent,struct,real,concAbs,import,relGoal,fullDev,oftMind,emotVal,tempOr
visImag,1.000000,0.155124,0.190250,0.392717,0.182357,0.172258,0.342683,0.351882,0.262818,0.334836,0.344044,-0.046350
inSpeech,0.155124,1.000000,0.162204,0.288172,0.313743,0.241900,0.186796,0.110923,0.147356,0.279936,0.035129,0.037519
intent,0.190250,0.162204,1.000000,0.402865,0.294887,0.348010,0.197543,0.176745,0.407325,0.064615,0.218919,0.234458
struct,0.392717,0.288172,0.402865,1.000000,0.498062,0.461555,0.551079,0.543600,0.595611,0.468592,0.373945,0.165864
real,0.182357,0.313743,0.294887,0.498062,1.000000,0.813847,0.639471,0.381467,0.426020,0.202108,0.247378,0.230118
concAbs,0.172258,0.241900,0.348010,0.461555,0.813847,1.000000,0.525021,0.216487,0.522466,0.137402,0.252095,0.156749
import,0.342683,0.186796,0.197543,0.551079,0.639471,0.525021,1.000000,0.647459,0.535839,0.472031,0.488881,0.162534
relGoal,0.351882,0.110923,0.176745,0.543600,0.381467,0.216487,0.647459,1.000000,0.458980,0.570866,0.370964,0.197132
fullDev,0.262818,0.147356,0.407325,0.595611,0.426020,0.522466,0.535839,0.458980,1.000000,0.382372,0.388905,0.065842
oftMind,0.334836,0.279936,0.064615,0.468592,0.202108,0.137402,0.472031,0.570866,0.382372,1.000000,0.367555,0.028734



Within-Participant Correlation Matrix:


,visImag,inSpeech,intent,struct,real,concAbs,import,relGoal,fullDev,oftMind,emotVal,tempOr
visImag,1.000000,-0.051376,0.044470,0.065694,0.045059,0.109855,0.069481,-0.013075,0.086488,-0.014027,0.117747,-0.007593
inSpeech,-0.051376,1.000000,0.015849,0.188349,0.085677,0.060297,0.152386,0.107372,0.061305,0.098000,0.020784,0.030142
intent,0.044470,0.015849,1.000000,0.157743,0.203429,0.193415,0.181178,0.166145,0.204088,0.095108,0.125618,-0.019289
struct,0.065694,0.188349,0.157743,1.000000,0.227769,0.252089,0.249090,0.226322,0.192884,0.080830,0.059240,-0.000277
real,0.045059,0.085677,0.203429,0.227769,1.000000,0.457259,0.416580,0.326728,0.286375,0.148378,0.149980,-0.044376
concAbs,0.109855,0.060297,0.193415,0.252089,0.457259,1.000000,0.363003,0.283383,0.324931,0.127229,0.122351,-0.074235
import,0.069481,0.152386,0.181178,0.249090,0.416580,0.363003,1.000000,0.381995,0.313772,0.283153,0.137878,-0.010791
relGoal,-0.013075,0.107372,0.166145,0.226322,0.326728,0.283383,0.381995,1.000000,0.265648,0.231914,0.140856,0.036324
fullDev,0.086488,0.061305,0.204088,0.192884,0.286375,0.324931,0.313772,0.265648,1.000000,0.134143,0.130807,-0.072693
oftMind,-0.014027,0.098000,0.095108,0.080830,0.148378,0.127229,0.283153,0.231914,0.134143,1.000000,0.058497,0.035425


Columns passed to CFA model: ['visImag', 'inSpeech', 'intent', 'struct', 'real', 'concAbs', 'import', 'relGoal', 'fullDev', 'oftMind', 'emotVal', 'tempOr']
--- SEMOPY Stats Output ---
       DoF  DoF Baseline        chi2  chi2 p-value  chi2 Baseline       CFI  \
Value   22            36  190.528067           0.0    2373.042507  0.927888   

            GFI      AGFI       NFI       TLI     RMSEA        AIC  \
Value  0.919711  0.868619  0.919711  0.881999  0.058362  45.830642   

              BIC    LogLik  
Value  177.360408  0.084679  
---------------------------
WITHIN-LEVEL CFA fit: {'n': 2250, 'chi2': 190.52806730301276, 'df': 22, 'p': 0.0, 'cfi': 0.9278883174796655, 'tli': 0.8819990649667253, 'rmsea': 0.05836198464694641, 'srmr': 'Not available'}
       lval           rval  Est. Std p-value
1  inSpeech         Format -0.227303       -
0   visImag         Format  0.227651       -
6   concAbs        Realism  0.648871     0.0
5      real        Realism  0.704423       -
2    import 